In [ ]:
import pandas as pd

import time
import sys
import os

root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.insert(0, root_path)

from src.function_cacher import FunctionCacher
from src.utils.compare import all_instance_of
from src.exceptions import StateNotFoundError

In [ ]:
called = dict(
    create_df = 0,
    add_one = 0
)

In [ ]:
import datetime as dt

# Try changing cache size, validity period
# ========================================
function_cacher = FunctionCacher(cache_size = 2, valid_for = dt.timedelta(seconds = 1))
try:
    function_cacher.load_cache(inplace = True, overwrite_loaded_cache_attributes = True)
except StateNotFoundError:
    pass

# function_cacher.clear()

print(function_cacher.cache.valid_for, function_cacher.cache._last_accessed)
function_cacher.cache.invalidate_old_data()
print(function_cacher.cache.valid_for, function_cacher.cache._last_accessed)

def compare_df(one, two):
    if all_instance_of(pd.DataFrame, one, two):

        if not (
            (len(one.index) == len(two.index))
            and (len(one.columns) == len(two.columns))
        ):
            return False
        
        return bool((one == two).all().all())

# try changing the body of the function
# =====================================
@function_cacher()
def create_df(rows = 10):
    time.sleep(1)
    called["create_df"] += 1
    rows = max(rows, 10)
    return pd.DataFrame(dict(values = range(rows)))

# compare_funcs is used to compare passed inputs and cached
# inputs. The inputs' arguments are passed to each of the funcs,
# which either refuse to compare (return None), or return a boolean
# denoting whether values compared equal. Here, pandas dataframes (df)
# cannot be compared simply by equating (==), so use a custom
# function.
@function_cacher(compare_funcs = [compare_df])
def add_one(df: pd.DataFrame):
    time.sleep(1)
    called["add_one"] += 1
    return df + 1

# easily check the current cached data
print(function_cacher.get_cached_data(create_df))
start_time = time.perf_counter_ns()
# try changing the passed argument
# ================================
df = create_df(rows = 13)
df["values"] = df["values"] + 1
df_add_one = add_one(df)
end_time = time.perf_counter_ns()
print(f"Time spent: {(end_time - start_time)/1e9}")
function_cacher.save()

# see how each function is called (is a cached value returned or 
# the function run properly)
print(called)
df_add_one
